In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:

import pandas as pd

data = pd.read_csv('/content/drive/My Drive/SentimentAnalysis/rnn/data/clean_data.csv', error_bad_lines=False)
data.CleanText = data.CleanText.astype(str)
data.head()

,ItemID,Sentiment,SentimentSource,SentimentText,CleanText
0,1,0,Sentiment140,is so sad for my APL frie...,is so sad for my apl friend
1,2,0,Sentiment140,I missed the New Moon trail...,i missed the new moon trailer
2,3,1,Sentiment140,omg its already 7:30 :O,omg its already 7 30 o
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...,omgaga im sooo im gunna cry i ve been at this ...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...,i think mi bf is cheating on me t t


In [32]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['CleanText'], 
                                                    data['Sentiment'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=data['Sentiment'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1420750,) (157862,) (1420750,) (157862,)


In [0]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [0]:
def labelize_tweets_ug(tweets, label):
  result = []
  prefix = label
  for i, t in zip(tweets.index, tweets):
    result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
  return result

all_x = pd.concat([x_train, x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [38]:
all_x_w2v[1]

TaggedDocument(words=['my', '6', 'yo', 'just', 'called', 'and', 'left', 'me', 'a', 'vm', 'he', 'said', 'i', 'love', 'you', 'mommy', 'and', 'it', 's', 'ok', 'to', 'come', 'pick', 'me', 'up', 'tomorrow', 'awww', 'now', 'mama', 'is', 'a', 'mess'], tags=['all_1009730'])

In [40]:
cores = multiprocessing.cpu_count()
# Continuous Bag Of Words
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 1578612/1578612 [00:00<00:00, 2102111.62it/s]


In [41]:
%%time
for epoch in range(30):
  model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
  model_ug_cbow.alpha -= 0.002
  model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 1578612/1578612 [00:00<00:00, 2125127.38it/s]


CPU times: user 31min 2s, sys: 10.7 s, total: 31min 13s
Wall time: 16min 44s


In [42]:
# Skip Gram
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 1578612/1578612 [00:00<00:00, 2309020.50it/s]


In [43]:
%%time
for epoch in range(30):
  model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
  model_ug_sg.alpha -= 0.002
  model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 1578612/1578612 [00:00<00:00, 2078750.08it/s]


CPU times: user 54min 42s, sys: 11.3 s, total: 54min 53s
Wall time: 28min 50s


In [0]:
model_ug_cbow.save('w2v_model_ug_cbow.word2vec')
model_ug_sg.save('w2v_model_ug_sg.word2vec')

In [66]:
from gensim.models import KeyedVectors

model_ug_cbow = KeyedVectors.load('/content/drive/My Drive/SentimentAnalysis/rnn/w2vmodels/w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('/content/drive/My Drive/SentimentAnalysis/rnn/w2vmodels/w2v_model_ug_sg.word2vec')

len(model_ug_cbow.wv.vocab.keys())

113945

In [68]:
import numpy as np

embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 113945 word vectors.


In [70]:
from keras.preprocessing.text import Tokenizer

MAX_NB_WORDS = 80000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data['CleanText'])

Using TensorFlow backend.


In [0]:
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [72]:
from keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 35
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)
padded_test_sequences = pad_sequences(test_sequences, maxlen=MAX_LENGTH)
padded_train_sequences.shape

(1420750, 35)

In [0]:
embed_size = 200
# maximum number of words kept after tokenization based on their word frequency
MAX_NB_WORDS = 80000

num_words = MAX_NB_WORDS
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in tokenizer.word_index.items():
  if i >= num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [79]:
embedding_matrix.shape

(80000, 200)

In [0]:
import numpy as np 
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, GRU, Bidirectional
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate

MAX_NB_WORDS = 80000

def get_rnn_model_with_word2vec():
  embedding_dim = 200
  inp = Input(shape=(MAX_LENGTH, ))
  x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=True)(inp)
  x = SpatialDropout1D(0.3)(x)
  x = Bidirectional(GRU(100, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)

  model = Model(inputs=inp, outputs=outp)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [82]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

rnn_model_with_word2vec = get_rnn_model_with_word2vec()

filepath="/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-word2vec-model-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

batch_size = 256
epochs = 4

history = rnn_model_with_word2vec.fit(x=padded_train_sequences, 
                    y=y_train, 
                    validation_data=(padded_test_sequences, y_test), 
                    batch_size=batch_size, 
                    callbacks=[checkpoint], 
                    epochs=epochs, 
                    verbose=1)

NameError: ignored

In [84]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

rnn_model_word2vec = load_model('/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-word2vec-model-03-0.8425.hdf5')

y_pred = rnn_model_word2vec.predict(padded_test_sequences, verbose=1, batch_size=2048)

y_pred = pd.DataFrame(y_pred, columns=['prediction'])
y_pred['prediction'] = y_pred['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
print('Accuracy of test set: ', accuracy_score(y_test, y_pred)*100, '%')

157862/157862 [==============================] - 7s 43us/step
Accuracy of test set:  84.25333519149638 %


In [0]:
# saving tokenizer
import pickle

with open('/content/drive/My Drive/SentimentAnalysis/rnn/tokenizers/tokenizer-rnn-word2vec.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)